#### MLQE XLMR

In [25]:
model_dir="/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers/TrainTeacher/Baseline.baseline/teacher_dir"

explainers_dirs = {
    "attention": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers/TrainStudent/KLDCoeff.5+Seed.9/teacher_expl_dir",
    "attention_softmax_trained": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true+NormalizeHeadCoeffs.softmax/teacher_expl_dir",
    "attention_sparsemax_trained": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true/teacher_expl_dir",
    "attention_entmax_trained": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true+NormalizeHeadCoeffs.entmax/teacher_expl_dir",
}

#explainers_dirs = {
#    "attention": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers/TrainStudent/KLDCoeff.5+Seed.9/teacher_expl_dir",
#    "topk_attention": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+TeacherNormalizerFn.topk_softmax/teacher_expl_dir",
#    "attention_trained": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true/teacher_expl_dir",
#    "topk_attention_trained": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true+Seed.11+TeacherNormalizerFn.topk_softmax/teacher_expl_dir",
#}

#explainers_dirs = {
#    "attention": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers/TrainStudent/KLDCoeff.5+Seed.9/teacher_expl_dir",
#    "attention_trained_1k": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true+NumExamples.1050/teacher_expl_dir",
#    "attention_trained_2k": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true+NumExamples.2100/teacher_expl_dir",
#    "attention_trained_4k": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true/teacher_expl_dir",
#    "attention_trained_8k": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true+NumExamples.8400/teacher_expl_dir",
#}

#explainers_dirs = {
#    "attention": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers/TrainStudent/KLDCoeff.5+Seed.9/teacher_expl_dir",
#    "attention_trained_lastlayers": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true/teacher_expl_dir",
#    "attention_trained_alllayers": "/projects/tir2/users/pfernand/metaexpl-experiments/mlqe-xlmr-xlmr-alllayers-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true/teacher_expl_dir",
#}

seed=0
max_len=256
batch_size=1
task="mlqe"
tokenizer="xlm-r"

#### IMDB ELECTRA

In [7]:
model_dir="/projects/tir2/users/pfernand/metaexpl-experiments/imdb-electra-electra/TrainTeacher/Baseline.baseline/teacher_dir"
explainers_dirs = {
    "attention": "/projects/tir2/users/pfernand/metaexpl-experiments/imdb-electra-electra-v2/TrainStudent/KLDCoeff.5/teacher_expl_dir",
    "attention_trained": "/projects/tir2/users/pfernand/metaexpl-experiments/imdb-electra-electra-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.2e-4+MetaLearn.true/teacher_expl_dir",
    "topk_attention_trained": "/projects/tir2/users/pfernand/metaexpl-experiments/imdb-electra-electra-v2/TrainStudent/KLDCoeff.5+MetaExplicit.true+MetaLR.5e-4+MetaLearn.true+TeacherNormalizerFn.topk_softmax/teacher_expl_dir"
}
seed=0
max_len=256
batch_size=1
task="imdb"
tokenizer="electra"

In [28]:
%load_ext autoreload
%autoreload 2
from meta_expl.models import load_model
from meta_expl.explainers import *
from meta_expl.utils import PRNGSequence

import jax.numpy as jnp

from transformers import ElectraTokenizerFast, BertTokenizerFast, XLMRobertaTokenizerFast

if task == "imdb":
    from meta_expl.data.imdb import dataloader, load_data
elif task == "mlqe":
    from meta_expl.data.mlqe import dataloader, load_data
elif task == "sst2":
    from meta_expl.data.sst2 import dataloader, load_data
else:
    raise ValueError(f"Unknown task {task}")
    
if tokenizer == "electra":
    tokenizer = ElectraTokenizerFast.from_pretrained(
        "google/electra-small-discriminator"
    )
elif tokenizer == "mbert":
    tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")
elif tokenizer == "xlm-r":
    tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
elif tokenizer == "xlm-r-large":
    tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-large")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import flax.nn as nn
from entmax_jax.activations import sparsemax, entmax15

rngseq = PRNGSequence(seed)

model, params, dummy_state = load_model(model_dir, batch_size=1)
input_ids = jnp.ones((batch_size, max_len), jnp.int32)
dummy_inputs = {
    "input_ids": input_ids,
    "attention_mask": jnp.ones_like(input_ids),
    "token_type_ids": jnp.arange(jnp.atleast_2d(input_ids).shape[-1]),
    "position_ids": jnp.ones_like(input_ids),
}

explainers = {}
for explainer_name, explainer_dir in explainers_dirs.items():
    if explainer_dir is None:
        explainers[explainer_name] = create_explainer(next(rngseq), dummy_inputs, dummy_state, explainer_type=explainer_name)
    else:
        explainers[explainer_name] = load_explainer(explainer_dir, dummy_inputs, dummy_state)
       
    print(explainers[explainer_name][0])
    if explainers[explainer_name][0].normalize_head_coeffs:
        if explainers[explainer_name][0].normalize_head_coeffs == "sparsemax":
            normalizer_fn = sparsemax
        elif explainers[explainer_name][0].normalize_head_coeffs == "entmax":
            normalizer_fn = entmax15
        else:
            normalizer_fn = nn.softmax
    else:
        normalizer_fn = lambda x: x
        
    print(f"{explainer_name}:", normalizer_fn(explainers[explainer_name][1]["params"]['head_coeffs']))

data = load_data("student", "test")

AttentionExplainer(
    # attributes
    aggregator_idx = 'mean'
    aggregator_dim = 'row'
    layer_idx = None
    init_fn = 'uniform'
    normalize_head_coeffs = 'sparsemax'
)
attention: [0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444 0.00694444
 0.006

In [20]:
dummy_state["attentions"][-1].shape

(1, 1, 1, 256)

In [15]:
from copy import deepcopy

num_heads = 4 * 12
def create_single_head_explainer(base_explainer, num_heads, head_no):
    modified = flax.core.unfreeze(base_explainer[1])
    head_coefs = [float("-inf") for _ in range(num_heads)]
    head_coefs[head_no] = 1
    modified["params"]["head_coeffs"] = jnp.array(head_coefs)
    explainer = deepcopy(base_explainer[0])
    explainer.layer_idx = base_explainer[0].layer_idx
    return (explainer, flax.core.freeze(modified))

heads_p = sparsemax(explainers["attention_trained"][1]["params"]['head_coeffs'])
for i in range(num_heads):
    if True: # heads_p[i] > 0:
        explainers[f"head_{i}"] = create_single_head_explainer(explainers["attention"], num_heads, i)

In [30]:
from IPython.display import Markdown, display

def print_coloured(subwords, weights, max_alpha=0.8):
    text = []
    colour = ["250, 133, 205" if w <= 0 else "135,206,250" for w in weights]
    weights = weights / jnp.max(weights)
    for i, subword in enumerate(subwords):
        if subword == "<s>":
            subword = "[CLS]"
        if subword == "</s>":
            subword = "[EOS]"
        if subword[0] == "▁":
            subword = subword[1:]
            if i:
                text.append(" ")
        colorstr = f"<span style='background-color:rgba({colour[i]},{weights[i] / max_alpha:.03f}'>{subword}</span>"
        text.append(colorstr)
    display(Markdown(" ".join(text)))

In [31]:
import jax
import jax.numpy as jnp
from functools import partial

@partial(jax.jit, static_argnums=(0,))
def get_out_state(model, params, x):
    """ Evaluation step """
    return model.apply(params, **x)

@partial(jax.jit, static_argnums=(0,))
def get_explanation(explainer, params, x, state):
    """ Evaluation step """
    return explainer.apply(
        params, x, state
    )[0]

num_samples = 20
for sample in data[:num_samples]:
    if task == "mlqe":
        label, inputs = sample["z_mean"], f"{sample['original']} </s> {sample['translation']}"
    if task == "imdb":
        label, inputs = sample['label'], sample['text']
    # extract token ids and subwords from tokenizer
    x = dict(tokenizer([inputs], truncation=True, padding='max_length', max_length=max_len, return_tensors="jax"))
    num_tokens = jnp.sum(jnp.where(x['input_ids'] != 0, 1, 0))
    bp_tokens = tokenizer.convert_ids_to_tokens(x['input_ids'][0][:num_tokens], skip_special_tokens=False)
    # extract model state 
    out, state = get_out_state(model, params, x)
    print(f"Label: {label}")
    print(f"Out: {out}\n")
    #print(state)
    for explainer, (expl_cls, expl_params) in explainers.items():
        print(f"Explainer: {explainer}")
        explanation = get_explanation(expl_cls, expl_params, x, state)[0][:num_tokens]
        
        if explainer == "attention_query_col":
            col_explanation = explanation
        if explainer == "attention_query_col_trained":
            col_trained_explanation = explanation
        #print(explanation)
        assert jnp.isclose(jnp.sum(explanation), 1, atol=1e-2), jnp.sum(explanation)
        print_coloured(bp_tokens, explanation)
    #print(col_trained_explanation - col_explanation)
    #print(f"Attention Diff")
    #print_coloured(bp_tokens, col_trained_explanation - col_explanation)
    print("----------")

Label: 0.8965325759587875
Out: [[0.9361073]]

Explainer: attention


<span style='background-color:rgba(135,206,250,0.037'>[CLS]</span>   <span style='background-color:rgba(135,206,250,0.003'>Este</span>   <span style='background-color:rgba(135,206,250,1.250'>important</span>   <span style='background-color:rgba(135,206,250,0.006'>să</span>   <span style='background-color:rgba(135,206,250,0.003'>se</span>   <span style='background-color:rgba(135,206,250,0.003'>stabil</span> <span style='background-color:rgba(135,206,250,0.003'>ească</span>   <span style='background-color:rgba(135,206,250,0.003'>o</span>   <span style='background-color:rgba(135,206,250,0.006'>legătură</span>   <span style='background-color:rgba(135,206,250,0.004'>între</span>   <span style='background-color:rgba(135,206,250,0.004'>client</span>   <span style='background-color:rgba(135,206,250,0.005'>și</span>   <span style='background-color:rgba(135,206,250,0.005'>angajat</span> <span style='background-color:rgba(135,206,250,0.008'>,</span>   <span style='background-color:rgba(135,206,250,0.004'>cunoscut</span> <span style='background-color:rgba(135,206,250,0.006'>ă</span>   <span style='background-color:rgba(135,206,250,0.006'>drept</span>   <span style='background-color:rgba(135,206,250,0.005'>management</span> <span style='background-color:rgba(135,206,250,0.004'>ul</span>   <span style='background-color:rgba(135,206,250,0.002'>rela</span> <span style='background-color:rgba(135,206,250,0.004'>ției</span>   <span style='background-color:rgba(135,206,250,0.003'>cu</span>   <span style='background-color:rgba(135,206,250,0.005'>clienți</span> <span style='background-color:rgba(135,206,250,0.003'>i</span> <span style='background-color:rgba(135,206,250,0.014'>.</span> <span style='background-color:rgba(135,206,250,0.011'>[EOS]</span>   <span style='background-color:rgba(135,206,250,0.003'>It</span>   <span style='background-color:rgba(135,206,250,0.003'>is</span>   <span style='background-color:rgba(135,206,250,0.005'>important</span>   <span style='background-color:rgba(135,206,250,0.005'>to</span>   <span style='background-color:rgba(135,206,250,0.005'>establish</span>   <span style='background-color:rgba(135,206,250,0.004'>a</span>   <span style='background-color:rgba(135,206,250,0.006'>connection</span>   <span style='background-color:rgba(135,206,250,0.004'>between</span>   <span style='background-color:rgba(135,206,250,0.003'>the</span>   <span style='background-color:rgba(135,206,250,0.004'>client</span>   <span style='background-color:rgba(135,206,250,0.003'>and</span>   <span style='background-color:rgba(135,206,250,0.002'>the</span>   <span style='background-color:rgba(135,206,250,0.006'>employee</span>   <span style='background-color:rgba(135,206,250,0.005'>known</span>   <span style='background-color:rgba(135,206,250,0.002'>as</span>   <span style='background-color:rgba(135,206,250,0.002'>the</span>   <span style='background-color:rgba(135,206,250,0.005'>management</span>   <span style='background-color:rgba(135,206,250,0.002'>of</span>   <span style='background-color:rgba(135,206,250,0.001'>the</span>   <span style='background-color:rgba(135,206,250,0.002'>customer</span>   <span style='background-color:rgba(135,206,250,0.003'>relationship</span> <span style='background-color:rgba(135,206,250,0.009'>.</span> <span style='background-color:rgba(135,206,250,0.020'>[EOS]</span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span>

Explainer: attention_softmax_trained


<span style='background-color:rgba(135,206,250,0.084'>[CLS]</span>   <span style='background-color:rgba(135,206,250,0.006'>Este</span>   <span style='background-color:rgba(135,206,250,1.250'>important</span>   <span style='background-color:rgba(135,206,250,0.012'>să</span>   <span style='background-color:rgba(135,206,250,0.006'>se</span>   <span style='background-color:rgba(135,206,250,0.005'>stabil</span> <span style='background-color:rgba(135,206,250,0.006'>ească</span>   <span style='background-color:rgba(135,206,250,0.006'>o</span>   <span style='background-color:rgba(135,206,250,0.011'>legătură</span>   <span style='background-color:rgba(135,206,250,0.007'>între</span>   <span style='background-color:rgba(135,206,250,0.008'>client</span>   <span style='background-color:rgba(135,206,250,0.011'>și</span>   <span style='background-color:rgba(135,206,250,0.009'>angajat</span> <span style='background-color:rgba(135,206,250,0.015'>,</span>   <span style='background-color:rgba(135,206,250,0.008'>cunoscut</span> <span style='background-color:rgba(135,206,250,0.012'>ă</span>   <span style='background-color:rgba(135,206,250,0.011'>drept</span>   <span style='background-color:rgba(135,206,250,0.010'>management</span> <span style='background-color:rgba(135,206,250,0.008'>ul</span>   <span style='background-color:rgba(135,206,250,0.004'>rela</span> <span style='background-color:rgba(135,206,250,0.008'>ției</span>   <span style='background-color:rgba(135,206,250,0.005'>cu</span>   <span style='background-color:rgba(135,206,250,0.010'>clienți</span> <span style='background-color:rgba(135,206,250,0.006'>i</span> <span style='background-color:rgba(135,206,250,0.025'>.</span> <span style='background-color:rgba(135,206,250,0.019'>[EOS]</span>   <span style='background-color:rgba(135,206,250,0.006'>It</span>   <span style='background-color:rgba(135,206,250,0.007'>is</span>   <span style='background-color:rgba(135,206,250,0.011'>important</span>   <span style='background-color:rgba(135,206,250,0.010'>to</span>   <span style='background-color:rgba(135,206,250,0.010'>establish</span>   <span style='background-color:rgba(135,206,250,0.007'>a</span>   <span style='background-color:rgba(135,206,250,0.012'>connection</span>   <span style='background-color:rgba(135,206,250,0.007'>between</span>   <span style='background-color:rgba(135,206,250,0.005'>the</span>   <span style='background-color:rgba(135,206,250,0.008'>client</span>   <span style='background-color:rgba(135,206,250,0.007'>and</span>   <span style='background-color:rgba(135,206,250,0.004'>the</span>   <span style='background-color:rgba(135,206,250,0.011'>employee</span>   <span style='background-color:rgba(135,206,250,0.009'>known</span>   <span style='background-color:rgba(135,206,250,0.005'>as</span>   <span style='background-color:rgba(135,206,250,0.005'>the</span>   <span style='background-color:rgba(135,206,250,0.009'>management</span>   <span style='background-color:rgba(135,206,250,0.004'>of</span>   <span style='background-color:rgba(135,206,250,0.003'>the</span>   <span style='background-color:rgba(135,206,250,0.005'>customer</span>   <span style='background-color:rgba(135,206,250,0.006'>relationship</span> <span style='background-color:rgba(135,206,250,0.019'>.</span> <span style='background-color:rgba(135,206,250,0.042'>[EOS]</span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span>

Explainer: attention_sparsemax_trained


...

Explainer: attention_sparsemax_trained


<span style='background-color:rgba(135,206,250,0.376'>[CLS]</span>   <span style='background-color:rgba(135,206,250,0.248'>Po</span> <span style='background-color:rgba(135,206,250,0.900'>ten</span> <span style='background-color:rgba(135,206,250,0.583'>ția</span> <span style='background-color:rgba(135,206,250,0.585'>lul</span>   <span style='background-color:rgba(135,206,250,0.867'>alimentar</span>   <span style='background-color:rgba(135,206,250,0.835'>a</span>   <span style='background-color:rgba(135,206,250,0.495'>spor</span> <span style='background-color:rgba(135,206,250,0.618'>it</span>   <span style='background-color:rgba(135,206,250,0.811'>creștere</span> <span style='background-color:rgba(135,206,250,0.631'>a</span>   <span style='background-color:rgba(135,206,250,0.633'>popula</span> <span style='background-color:rgba(135,206,250,1.188'>ției</span> <span style='background-color:rgba(135,206,250,0.633'>,</span>   <span style='background-color:rgba(135,206,250,0.610'>cele</span>   <span style='background-color:rgba(135,206,250,0.656'>mai</span>   <span style='background-color:rgba(135,206,250,0.565'>mari</span>   <span style='background-color:rgba(135,206,250,0.526'>den</span> <span style='background-color:rgba(135,206,250,0.528'>s</span> <span style='background-color:rgba(135,206,250,0.884'>ități</span>   <span style='background-color:rgba(135,206,250,0.903'>fiind</span>   <span style='background-color:rgba(135,206,250,0.570'>at</span> <span style='background-color:rgba(135,206,250,0.454'>in</span> <span style='background-color:rgba(135,206,250,0.542'>se</span>   <span style='background-color:rgba(135,206,250,1.250'>în</span>   <span style='background-color:rgba(135,206,250,0.436'>Mex</span> <span style='background-color:rgba(135,206,250,0.410'>ic</span>   <span style='background-color:rgba(135,206,250,1.000'>și</span>   <span style='background-color:rgba(135,206,250,0.475'>Peru</span> <span style='background-color:rgba(135,206,250,0.949'>.</span> <span style='background-color:rgba(135,206,250,0.747'>[EOS]</span>   <span style='background-color:rgba(135,206,250,0.696'>Food</span>   <span style='background-color:rgba(135,206,250,0.751'>potential</span>   <span style='background-color:rgba(135,206,250,0.687'>has</span>   <span style='background-color:rgba(135,206,250,0.831'>increased</span>   <span style='background-color:rgba(135,206,250,0.709'>population</span>   <span style='background-color:rgba(135,206,250,0.734'>growth</span> <span style='background-color:rgba(135,206,250,0.636'>,</span>   <span style='background-color:rgba(135,206,250,0.818'>with</span>   <span style='background-color:rgba(135,206,250,0.706'>the</span>   <span style='background-color:rgba(135,206,250,0.814'>highest</span>   <span style='background-color:rgba(135,206,250,0.543'>den</span> <span style='background-color:rgba(135,206,250,0.564'>s</span> <span style='background-color:rgba(135,206,250,0.555'>ities</span>   <span style='background-color:rgba(135,206,250,0.717'>being</span>   <span style='background-color:rgba(135,206,250,0.855'>reached</span>   <span style='background-color:rgba(135,206,250,0.630'>in</span>   <span style='background-color:rgba(135,206,250,0.527'>Mexico</span>   <span style='background-color:rgba(135,206,250,0.715'>and</span>   <span style='background-color:rgba(135,206,250,0.393'>Peru</span> <span style='background-color:rgba(135,206,250,0.911'>.</span> <span style='background-color:rgba(135,206,250,0.786'>[EOS]</span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span>

Explainer: attention_entmax_trained


<span style='background-color:rgba(135,206,250,0.601'>[CLS]</span>   <span style='background-color:rgba(135,206,250,0.167'>Po</span> <span style='background-color:rgba(135,206,250,1.250'>ten</span> <span style='background-color:rgba(135,206,250,0.220'>ția</span> <span style='background-color:rgba(135,206,250,0.213'>lul</span>   <span style='background-color:rgba(135,206,250,0.312'>alimentar</span>   <span style='background-color:rgba(135,206,250,0.268'>a</span>   <span style='background-color:rgba(135,206,250,0.168'>spor</span> <span style='background-color:rgba(135,206,250,0.201'>it</span>   <span style='background-color:rgba(135,206,250,0.235'>creștere</span> <span style='background-color:rgba(135,206,250,0.203'>a</span>   <span style='background-color:rgba(135,206,250,0.173'>popula</span> <span style='background-color:rgba(135,206,250,0.430'>ției</span> <span style='background-color:rgba(135,206,250,0.235'>,</span>   <span style='background-color:rgba(135,206,250,0.191'>cele</span>   <span style='background-color:rgba(135,206,250,0.200'>mai</span>   <span style='background-color:rgba(135,206,250,0.215'>mari</span>   <span style='background-color:rgba(135,206,250,0.176'>den</span> <span style='background-color:rgba(135,206,250,0.161'>s</span> <span style='background-color:rgba(135,206,250,0.290'>ități</span>   <span style='background-color:rgba(135,206,250,0.240'>fiind</span>   <span style='background-color:rgba(135,206,250,0.182'>at</span> <span style='background-color:rgba(135,206,250,0.139'>in</span> <span style='background-color:rgba(135,206,250,0.188'>se</span>   <span style='background-color:rgba(135,206,250,0.391'>în</span>   <span style='background-color:rgba(135,206,250,0.130'>Mex</span> <span style='background-color:rgba(135,206,250,0.118'>ic</span>   <span style='background-color:rgba(135,206,250,0.360'>și</span>   <span style='background-color:rgba(135,206,250,0.134'>Peru</span> <span style='background-color:rgba(135,206,250,0.361'>.</span> <span style='background-color:rgba(135,206,250,0.272'>[EOS]</span>   <span style='background-color:rgba(135,206,250,0.240'>Food</span>   <span style='background-color:rgba(135,206,250,0.274'>potential</span>   <span style='background-color:rgba(135,206,250,0.261'>has</span>   <span style='background-color:rgba(135,206,250,0.253'>increased</span>   <span style='background-color:rgba(135,206,250,0.199'>population</span>   <span style='background-color:rgba(135,206,250,0.231'>growth</span> <span style='background-color:rgba(135,206,250,0.223'>,</span>   <span style='background-color:rgba(135,206,250,0.249'>with</span>   <span style='background-color:rgba(135,206,250,0.214'>the</span>   <span style='background-color:rgba(135,206,250,0.263'>highest</span>   <span style='background-color:rgba(135,206,250,0.166'>den</span> <span style='background-color:rgba(135,206,250,0.165'>s</span> <span style='background-color:rgba(135,206,250,0.186'>ities</span>   <span style='background-color:rgba(135,206,250,0.210'>being</span>   <span style='background-color:rgba(135,206,250,0.286'>reached</span>   <span style='background-color:rgba(135,206,250,0.222'>in</span>   <span style='background-color:rgba(135,206,250,0.151'>Mexico</span>   <span style='background-color:rgba(135,206,250,0.221'>and</span>   <span style='background-color:rgba(135,206,250,0.107'>Peru</span> <span style='background-color:rgba(135,206,250,0.390'>.</span> <span style='background-color:rgba(135,206,250,0.410'>[EOS]</span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span> <span style='background-color:rgba(250, 133, 205,0.000'><pad></span>

----------


In [32]:
from entmax_jax.losses import softmax_loss

logits = jnp.array[[1, 3, 2, -1, 10]]

def topk_softmax(logits, topk=0.1, axis=-1):
    threshold = -1e-7
    numtoks = jnp.sum(jnp.where(logits > threshold, 1, 0), axis=axis)
    topk_numtoks = jnp.maximum(jnp.round(numtoks * topk), 1)
    unnorm_probs = jnp.where(jnp.argsort(logits, axis=axis) < topk_numtoks[:, None], 1, 0)
    
    return unnorm_probs / jnp.sum(unnorm_probs, axis=axis, keepdims=True)
    

expl_cls, expl_params = explainers["attention_explainer"]
def loss(state):
    expl = get_explanation(expl_cls, expl_params, x, state)
    return softmax_loss(expl, gold, mask=x["attention_mask"])

print(loss(state))
print(jax.grad(loss)(state)["attentions"][-1][0])

TypeError: 'function' object is not subscriptable

In [102]:
from entmax_jax.losses import softmax_loss

logits = jnp.array([[1., 3., 2., -1., 10.], [-1, 10, 2, 3, 4]])

def batched_scatter(input, index, src):
    """ 2-d batched scatter """
    idx = jnp.arange(input.shape[0]).reshape((-1, 1))
    idx = jnp.repeat(idx, input.shape[1], 1)
    return input.at[idx, index].set(src)

def topk_softmax(logits, topk=0.1, temperature=1., axis=-1):
    #TODO: currently only works for 2-d tensors with axis=-1
    threshold = -1e7
    numtoks = jnp.sum(jnp.where(logits > threshold, 1, 0), axis=axis)
    topk_numtoks = jnp.maximum(jnp.round(numtoks * topk), 1)
    
    #obtain ranks from argsort
    sorted_idx = jnp.argsort(logits, axis=axis)[:,::-1]
    zeros = jnp.zeros(logits.shape, sorted_idx.dtype)
    ranges = jnp.repeat(jnp.arange(logits.shape[-1]).reshape((1,-1)), logits.shape[0], 0)
    ranks = batched_scatter(zeros, sorted_idx, ranges)
    
    topk_logits = jnp.where(ranks < topk_numtoks[:, None], logits * temperature, threshold)
    return nn.softmax(topk_logits, axis=axis)

def loss(logits):
    return jnp.sum(topk_softmax(logits, topk=0.5)**2, axis=1).mean()

print(topk_softmax(logits, topk=0.5, temperature=0.5))
jax.grad(loss)(logits)

[[0.         0.02931223 0.         0.         0.97068775]
 [0.         0.95257413 0.         0.         0.04742587]]


DeviceArray([[ 0.        , -0.00090856,  0.        ,  0.        ,
               0.00090849],
             [ 0.        ,  0.0024544 ,  0.        ,  0.        ,
              -0.00245431]], dtype=float32)